# Example of retrieving MERRA-2 data, computing running mean (weekly) and climatology

Outputs of this notebook are inputs for the MERRA-2/SIRS model. 

In [1]:
import xarray as xr
from tqdm.notebook import tqdm
import numpy as np

In [16]:
# MERRA-2 data location on tigress machine
# daily MERRA-2 data: Jan 1980-Aug 2018

ifile = '/tigress/samueltb/MERRA2/Daily/merged/QV2M/MERRA2.tavg1_2d_slv_Nx.198001-201808.nc'
q = xr.open_dataset(ifile).rename(lat='latitude', lon='longitude')['QV2M']

## create (daily) climatology

In [19]:
q

<xarray.DataArray 'QV2M' (time: 14123, latitude: 361, longitude: 576)>
[2936680128 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01T12:00:00 ... 2018-08-31T12:00:00
  * longitude  (longitude) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * latitude   (latitude) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
Attributes:
    standard_name:   2-meter_specific_humidity
    long_name:       2-meter_specific_humidity
    units:           kg kg-1
    fmissing_value:  1000000000000000.0
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0

In [25]:
climatology = q.groupby('time.dayofyear').mean()

In [26]:
climatology.to_netcdf(f'./SIRSmodel_input/climatology.MERRA2.1980-2018.QV2M.daily.nc')

## get data 1981-2017

In [12]:
year_start = 1981
year_end = 2017

day_start = f'{year_start}-01-01'
day_end = f'{year_end}-12-31'

In [10]:
q = q.sel(time=slice(day_start,day_end))

In [11]:
q

<xarray.DataArray 'QV2M' (time: 13514, latitude: 361, longitude: 576)>
[2810047104 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01T12:00:00 ... 2017-12-31T12:00:00
  * longitude  (longitude) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * latitude   (latitude) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
Attributes:
    standard_name:   2-meter_specific_humidity
    long_name:       2-meter_specific_humidity
    units:           kg kg-1
    fmissing_value:  1000000000000000.0
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0

In [14]:
q.to_netcdf(f'./SIRSmodel_input/MERRA2.{year_start}-{year_end}.QV2M.daily.nc')

In [15]:
q

<xarray.DataArray 'QV2M' (time: 13514, latitude: 361, longitude: 576)>
[2810047104 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01T12:00:00 ... 2017-12-31T12:00:00
  * longitude  (longitude) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * latitude   (latitude) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
Attributes:
    standard_name:   2-meter_specific_humidity
    long_name:       2-meter_specific_humidity
    units:           kg kg-1
    fmissing_value:  1000000000000000.0
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0

# make weekly running mean

### climatology

In [49]:
climatology = xr.open_dataarray(f'./SIRSmodel_input/climatology.MERRA2.1980-2018.QV2M.daily.nc')

In [50]:
clim_week = climatology.rolling(dayofyear=7,center=True).mean().dropna('dayofyear')
clim_week

<xarray.DataArray 'QV2M' (dayofyear: 360, latitude: 361, longitude: 576)>
array([[[0.00044518, 0.00044518, 0.00044518, ..., 0.00044518,
         0.00044518, 0.00044518],
        [0.00043386, 0.00043395, 0.00043404, ..., 0.00043361,
         0.00043369, 0.00043378],
        [0.00042537, 0.00042557, 0.00042577, ..., 0.00042477,
         0.00042497, 0.00042517],
        ...,
        [0.00048209, 0.0004819 , 0.00048171, ..., 0.00048266,
         0.00048247, 0.00048228],
        [0.00048627, 0.00048619, 0.0004861 , ..., 0.00048654,
         0.00048645, 0.00048636],
        [0.0004895 , 0.0004895 , 0.0004895 , ..., 0.0004895 ,
         0.0004895 , 0.0004895 ]],

       [[0.00044142, 0.00044142, 0.00044142, ..., 0.00044142,
         0.00044142, 0.00044142],
        [0.00043063, 0.00043072, 0.00043081, ..., 0.00043038,
         0.00043046, 0.00043054],
        [0.00042321, 0.00042341, 0.00042361, ..., 0.00042261,
         0.00042281, 0.00042301],
...
        [0.00051974, 0.00051955, 0.00051935, ..., 0.00052035,
         0.00052015, 0.00051994],
        [0.00052951, 0.00052941, 0.00052932, ..., 0.00052979,
         0.0005297 , 0.0005296 ],
        [0.00053681, 0.00053681, 0.00053681, ..., 0.00053681,
         0.00053681, 0.00053681]],

       [[0.00046612, 0.00046612, 0.00046612, ..., 0.00046612,
         0.00046612, 0.00046612],
        [0.00045054, 0.00045062, 0.00045071, ..., 0.00045031,
         0.00045038, 0.00045046],
        [0.00043584, 0.00043604, 0.00043624, ..., 0.00043526,
         0.00043545, 0.00043564],
        ...,
        [0.00053013, 0.00053004, 0.00052995, ..., 0.00053042,
         0.00053032, 0.00053022],
        [0.0005421 , 0.00054203, 0.00054197, ..., 0.00054229,
         0.00054223, 0.00054216],
        [0.00055271, 0.00055271, 0.00055271, ..., 0.00055271,
         0.00055271, 0.00055271]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * latitude   (latitude) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * dayofyear  (dayofyear) int64 4 5 6 7 8 9 10 ... 357 358 359 360 361 362 363
Attributes:
    standard_name:   2-meter_specific_humidity
    long_name:       2-meter_specific_humidity
    units:           kg kg-1
    fmissing_value:  1000000000000000.0
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0

In [51]:
clim_week.to_netcdf(f'./SIRSmodel_input/climatology.MERRA2.1980-2018.QV2M.weekly.nc')

### data 1981-2017

In [2]:
da = xr.open_dataarray(f'./SIRSmodel_input/MERRA2.1981-2017.QV2M.daily.nc')

In [3]:
# 2m 20s
da_week = da.rolling(time=7,center=True).mean().dropna('time')

In [4]:
da_week

<xarray.DataArray 'QV2M' (time: 13508, latitude: 361, longitude: 576)>
array([[[0.00044131, 0.00044131, 0.00044131, ..., 0.00044131,
         0.00044131, 0.00044131],
        [0.00043596, 0.00043611, 0.00043625, ..., 0.00043553,
         0.00043567, 0.00043582],
        [0.00043836, 0.00043871, 0.00043906, ..., 0.00043727,
         0.00043764, 0.000438  ],
        ...,
        [0.00029216, 0.00029226, 0.00029237, ..., 0.00029184,
         0.00029195, 0.00029205],
        [0.00029257, 0.0002926 , 0.00029265, ..., 0.00029243,
         0.00029248, 0.00029252],
        [0.00029277, 0.00029277, 0.00029277, ..., 0.00029277,
         0.00029277, 0.00029277]],

       [[0.00041998, 0.00041998, 0.00041998, ..., 0.00041998,
         0.00041998, 0.00041998],
        [0.00040844, 0.00040851, 0.00040857, ..., 0.00040825,
         0.00040831, 0.00040838],
        [0.00040872, 0.00040899, 0.00040926, ..., 0.00040787,
         0.00040816, 0.00040844],
...
        [0.00078351, 0.00078416, 0.0007848 , ..., 0.00078151,
         0.00078217, 0.00078285],
        [0.00077536, 0.0007757 , 0.00077607, ..., 0.0007743 ,
         0.00077464, 0.000775  ],
        [0.00077289, 0.00077289, 0.00077289, ..., 0.00077289,
         0.00077289, 0.00077289]],

       [[0.00059352, 0.00059352, 0.00059352, ..., 0.00059352,
         0.00059352, 0.00059352],
        [0.00056848, 0.00056859, 0.00056873, ..., 0.00056812,
         0.00056824, 0.00056836],
        [0.00053801, 0.00053823, 0.00053847, ..., 0.00053739,
         0.00053759, 0.00053781],
        ...,
        [0.00083005, 0.0008305 , 0.00083095, ..., 0.00082868,
         0.00082912, 0.00082959],
        [0.0008359 , 0.00083616, 0.00083645, ..., 0.00083509,
         0.00083535, 0.00083562],
        [0.00084316, 0.00084316, 0.00084316, ..., 0.00084316,
         0.00084316, 0.00084316]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-04T12:00:00 ... 2017-12-28T12:00:00
  * longitude  (longitude) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * latitude   (latitude) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
Attributes:
    standard_name:   2-meter_specific_humidity
    long_name:       2-meter_specific_humidity
    units:           kg kg-1
    fmissing_value:  1000000000000000.0
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0

In [5]:
# 11s
da_week.to_netcdf(f'./SIRSmodel_input/MERRA2.1981-2017.QV2M.weekly.nc')